<a href="https://colab.research.google.com/github/theSubsurfaceGuy/AGU-mentoring-program/blob/master/Gassmann_Fluid_Substitution_Patchy_Saturation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reference papers to follow along the code 

Kumar, D., 2006, A tutorial on Gassmann Fluid Substitution: Formulation, Algorithm and Matlab code: Geohorizons, 11, 4-12

Dvorkin etal, 2001, Identifying patchy saturations from well log


In [ ]:
import ipywidgets as wg
from IPython.display import display

In [ ]:
rho_o=wg.IntSlider(description='Oil gravity:', max=60)
GOR=wg.IntSlider(description='GOR:', max=500)
rho_g=wg.FloatSlider(description='Gas gravity:', max=1)
T=wg.IntSlider(description='Temp (C):', max=500)
P=wg.IntSlider(description='Pres(psi):', max=5000)
S=wg.IntSlider(description='Salinity(ppm):', max=6000)
phi=wg.FloatSlider(description='Porosity:', max=1)
vsh=wg.FloatSlider(description='Shale Fraction:', max=1)
isw=wg.FloatSlider(description='Sw initial:', max=1)
tsw=wg.FloatSlider(description='Sw target:', max=1)
ifluid=wg.Dropdown( options=[('Oil', 1), ('Gas', 2)],description='Initial HC:')
fluid=wg.Dropdown( options=[('Brine', 1), ('Oil', 2), ('Gas', 2)],description='DesiredFluid')
vp=wg.IntSlider(description='Init Vp:', max=15000)
vs=wg.IntSlider(description='Init Vs:', max=10000)
rho=wg.FloatSlider(description='Density init:', max=8)
display(rho_o,GOR, rho_g,T,P,S,phi,Vsh, isw,tsw,ifluid, fluid,vp,vs,rho)

IntSlider(value=0, description='Oil gravity:', max=60)

IntSlider(value=0, description='GOR:', max=500)

FloatSlider(value=0.0, description='Gas gravity:', max=1.0)

IntSlider(value=0, description='Temp (C):', max=500)

IntSlider(value=0, description='Pres(psi):', max=5000)

...

Dropdown(description='Initial HC:', options=(('Oil', 1), ('Gas', 2)), value=1)

Dropdown(description='DesiredFluid', options=(('Brine', 1), ('Oil', 2), ('Gas', 2)), value=1)

IntSlider(value=0, description='Init Vp:', max=15000)

IntSlider(value=0, description='Init Vs:', max=10000)

FloatSlider(value=0.0, description='Density init:', max=8.0)

In [ ]:
rho_o = rho_o.value        #Oil gravity (deg API)
GOR = GOR.value            #GOR (L/L)
rho_g = rho_g.value        #Gas gravity (API)
T = T.value                #Temperature (0 C)
P = P.value                #Pressure (psi)
S = S.value                #water salinity (ppm)
phi = phi.value            #porosity (in fraction)
vsh = vsh.value            #Vsh (volume shale in fraction)
isw = isw.value            #initial water saturation (SW)
tsw = tsw.value            #target water saturation (in fraction)
ifluid = ifluid.value      #initial hydrocarbon is 1(oil), 2(gas)
fluid = fluid.value        #Desired fluid is 1(brine), 2(oil) 3(gas)
vp = vp.value              #ft/s - from log (initial value)
vs = vs.value              #ft/s - from log (initial value)
rho = rho.value            #gm/c - from log (initial value)

Say these values are selected by the GUI


In [ ]:
rho_o = 42        #Oil gravity (deg API)
GOR = 160.0       #GOR (L/L)
rho_g = 0.9       #Gas gravity (API)
T = 150.00        #Temperature (0 C)
P = 3200.00       #Pressure (psi)
S = 3800          #water salinity (ppm)
phi = 0.20        #porosity (in fraction)
vsh = 0.20        #Vsh (volume shale in fraction)
isw = 0.40        #initial water saturation (SW)
tsw = 1.00        #target water saturation (in fraction)
ifluid = 1        #initial hydrocarbon is 1(oil), 2(gas)
fluid = 3         #Desired fluid is 1(brine), 2(oil) 3(gas)
vp = 11000.0      #ft/s - from log (initial value)
vs = 6500.0       #ft/s - from log (initial value)
rho = 2.2         #gm/c - from log (initial value)

#Fixed parameters (e.g., Mavko et al., 1998)


In [ ]:
k_clay = 20.9    #Bulk mod (GPa)
k_qtz = 36.6
rho_clay = 2.58  #gm/cc
rho_qtz = 2.65

#Some applied properties

In [ ]:
div_mill = 1/1000000   #factor used to divide by million
fs2kms = 0.000305      #factor for ft/s to km/s conversion
kms2fs = 3280.84       #factor for km/s to ft/s conversion
v_clay = vsh*0.70      #Assumption: V_clay = 70% of VSH
v_qtz= 1-v_clay        #quartz fraction in mineral
ish = 1-isw            #initial hydrocarbon saturation
tsh = 1-tsw            #final hydrocarbon saturation
rho_o = 141.5/(rho_o+131.5) #oil gravity in gm/cc (from API)
P = P*6.894757*0.001         #Press in MPa (from Psi)
S = S*div_mill               #salinity as weight fraction
vp = vp*fs2kms               #ft/s to km/s
vs = vs*fs2kms               #ft/s to km/s

#Step 1: Matrix properties (using VRH averaging, equation 6)


In [ ]:
k_voigt = v_clay*k_clay + v_qtz*k_qtz
k_reuss = 1/(v_clay/k_clay + v_qtz/k_qtz)
k_matrix = 0.5*(k_voigt + k_reuss)                            #GPa
rho_matrix = v_clay*rho_clay+v_qtz*rho_qtz                    #gm/cc

In [ ]:
import numpy as np
w=np.zeros((5,4),dtype=float)

#Step 2: water/brine properties (Equations 10 and 11)


In [ ]:
w[0,0] = 1402.85 
w[0,2] = 3.437*10**(-3)    #Table 2
w[1,0] = 4.871
w[1,2] = 1.739*10**(-4)
w[2,0] = -0.04783
w[2,2] = -2.135*10**(-6)
w[3,0] = 1.487*10**(-4)
w[3,2] = -1.455*10**(-8)
w[4,0] = -2.197*10**(-7)
w[4,2] = 5.230*10**(-11)
w[0,1] = 1.524
w[0,3] = -1.197*10**(-5)
w[1,1] = -0.0111
w[1,3] = -1.628*10**(-6)
w[2,1] = 2.747*10**(-4)
w[2,3] = 1.237*10**(-8)
w[3,1] = -6.503*10**(-7)
w[3,3] = 1.327*10**(-10)
w[4,1] = 7.987*10**(-10)
w[4,3] = -4.614*10**(-13)

In [ ]:
sum = 0
for i in range(5):
  for j in range(4):
    sum= sum+ w[i,j]*np.power(T,i-1)*np.power(P,j-1)
v_water = sum

In [ ]:
v1 = 1170-9.6*T+(0.055*T*T)-8.5*(10**(-5))*T*T*T+2.6*P-0.0029*T*P-0.0476*P*P

In [ ]:
v_brine = v_water+S*v1+(S**1.5)*(780-10*P+0.16*P*P)-1820*S*S #m/s

In [ ]:
r1 = 489*P-2*T*P+0.016*T*T*P-1.3*(10**(-5))*T*T*T*P-0.333*P*P-0.002*T*P*P

In [ ]:
rho_water=1+(10**(-6))*(-80*T-3.3*T*T+0.00175*T*T*T+r1)

In [ ]:
r2 = 300*P-2400*P*S+T*(80+3*T-3300*S-13*P+47*P*S)

In [ ]:
rho_brine = rho_water+0.668*S+0.44*S*S+(10**(-6))*S*r2 #gm/cc (held const)

In [ ]:
k_brine = rho_brine*v_brine*v_brine*div_mill #GPa (held const)

#Step 3: Initial Hydrocarbon properties


In [ ]:
import math
if ifluid == 1: #'Oil’ Oil by default contains gas also4
  B0 = 0.972+0.00038*((2.495*GOR*math.sqrt(rho_g/rho_o)+T+17.8)**1.175)
  rho_ps = rho_o/((1+0.001*GOR)*B0)
  rho_s = (rho_o+0.0012*GOR*rho_g)/B0
  r1 = rho_s+(0.00277*P-1.71*0.0000001*P*P*P)*((rho_s-1.15)**2)+3.49*0.0001*P
  rho_hyc = r1/(0.972+3.81*0.0001*((T+17.78)**1.175))                                   #gm/cc (will change)
  v = 2096*math.sqrt(rho_ps/(2.6-rho_ps))-3.7*T+4.64*P+0.0115*(math.sqrt(18.33/rho_ps-16.97)-1)*T*P
  k_hyc = rho_hyc*v*v*div_mill  #GPa (will change)

else:
  R = 8.314 #gas means no OIL only gas is present, gas constant (eqn, same as in step7 for fluid == 3)
  Ta = T+273.15
  Ppr = P/(4.892-0.4048*rho_g)
  Tpr = Ta/(94.72+170.75*rho_g)
  E1 = exp((-Ppr**1.2)/(Tpr*(0.45+8*(0.56-1/Tpr)**2)))
  E = 0.109*((3.85-Tpr)**2)*E1
  Z1 = 0.03+0.00527*((3.5-Tpr)**3)
  Z = Z1*Ppr+(0.642*Tpr-0.007*(Tpr**4)-0.52)+E
  rho_hyc = 28.8*rho_g*P/(Z*R*Ta)
  dz_dp = Z1+0.109*(3.85-Tpr)^2*E1*(-1.2*Ppr^0.2/Tpr*(0.45+8*(0.56-1/Tpr)**2))
  yo = 0.85+5.6/(Ppr+2)+27.1/(Ppr+3.5)^2-8.7*exp(-0.65*(Ppr+1))
  k_hyc = P*yo/1000*1.0/(1-Ppr/Z*dz_dp)                                               ###GPa

#Step 4: Fluid properties(initial insitu model)

For patchy saturation using equation 5,6 from DVORKIN et al 2001,

In [ ]:
k_fl=isw*k_brine+(1-isw)*k_hyc #Equation 5
rho_fl = isw*rho_brine+ish*rho_hyc

In [ ]:
#K_fl = (k_brine - k_hyc) * Sw**e + k_hyc #Equation 6

#where e is a free parameter. For e = 1, Equation becomes Voigt average. For a large e the velocity is close to that for homogeneous saturation 

#Step 5: Insitu original moduli (for saturated – insitu rock, equations 4 and 5)


In [ ]:
dens_poros = 0 #1 (use porosity to est initial density), 0 (use input log)
if dens_poros == 1:
  rho = phi*rho_fl + (1-phi)*rho_matrix

k_sat = rho*(vp*vp-vs*vs*4/3)     #GPa (will change in step 9)
g = rho*vs*vs                     #GPa (held constant)

#Step 6: Porous frame properties (rewrite Gassmann eqn, equation 36)

In [ ]:
k1 = k_sat*(phi*k_matrix/k_fl+1-phi)-k_matrix
k2 = phi*k_matrix/k_fl+k_sat/k_matrix-1-phi
k_frame = k1/k2 #GPa (held constant)

#Step 7: select the type of output fluid, cal hyc/fluid prop (equations 32 to 35)


In [ ]:

if fluid == 1: #Brine
  print("Changing fluid to brine") 

if fluid == 2:     #Oil
  print("Changing fluid to Oil [with dissolved gas] with TWS brine")
  B0 = 0.972+0.00038*(2.495*GOR*math.sqrt(rho_g/rho_o)+T+17.8)**1.175
  rho_ps = rho_o/((1+0.001*GOR)*B0)
  rho_s = (rho_o+0.0012*GOR*rho_g)/B0
  r1 = rho_s+(0.00277*P-1.71*0.0000001*P*P*P)*((rho_s-1.15)**2)+3.49*0.0001*P
  rho_hyc = r1/(0.972+3.81*0.0001*((T+17.78)**1.175))                                     #gm/cc (will change)
  v = 2096*math.sqrt(rho_ps/(2.6-rho_ps))-3.7*T+4.64*P+0.0115*(math.sqrt(18.33/rho_ps-16.97)-1)*T*P
  k_hyc = rho_hyc*v*v*div_mill #GPa (will change)

if fluid == 3: #Gas
  print("Changing fluid to Gas with TWS brine")
  R = 8.314
  Ta = T+273.15
  Ppr = P/(4.892-0.4048*rho_g)
  Tpr = Ta/(94.72+170.75*rho_g)
  E1 = math.exp(-Ppr**1.2/Tpr*(0.45+8*(0.56-1/Tpr)**2))
  E = 0.109*((3.85-Tpr)**2)*E1
  Z1 = 0.03+0.00527*((3.5-Tpr)**3)
  Z = Z1*Ppr+0.642*Tpr-0.007*(Tpr**4)-0.52+E
  rho_hyc = 28.8*rho_g*P/(Z*R*Ta)
  dz_dp=Z1+0.109*((3.85-Tpr)**2)*E1*(-1.2*Ppr**0.2/Tpr*(0.45+8*(0.56-1/Tpr)**2))
  yo = 0.85+5.6/(Ppr+2)+27.1/(Ppr+3.5)**2-8.7*math.exp(-0.65*(Ppr+1))
  k_hyc = P*yo/1000*1.0/(1-Ppr/Z*dz_dp)                                                       #GPa

Changing fluid to Gas with TWS brine


#Step 8: Fluid properties (target saturation) and saturated rock density (equations 30 and 31)


In [ ]:

k_fl = 1/(tsw/k_brine + tsh/k_hyc)
rho_fl = tsw*rho_brine + tsh*rho_hyc
rho_sat = phi*rho_fl+(1-phi)*rho_matrix

In [ ]:
rho_sat

2.2987991301334105

#Step 9: Gassmann Saturated bulk modulus (equation 3)


In [ ]:

k1 = phi/k_fl+(1-phi)/k_matrix-k_frame/(k_matrix*k_matrix)
k_sat_new = k_frame + ((1-k_frame/k_matrix)**2)/k1

#Step 10: Vp, Vs, rho after fluid substitution (equations 1 and 2)


In [ ]:
vp_sat = math.sqrt((k_sat_new+g*4/3)/rho_sat)*kms2fs
vs_sat = math.sqrt(g/rho_sat)*kms2fs

In [ ]:
rho_sat

2.2987991301334105

In [ ]:
vp_sat

10714.077698124196

In [ ]:
vs_sat

6362.958277961363